In [ ]:
# Установка библиотек
%pip install bs4 openpyxl tqdm \
             multiprocess pandas numpy \
             openpyxl scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Импорт библиотек
import re
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import requests as rq
from IPython import display
from tqdm import tqdm
from multiprocess import Pool
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
lenta_ru_df = pd.read_csv("drive/MyDrive/lenta_arts.csv")
lenta_ru_df = lenta_ru_df.drop(columns=['docid', 'domain', 'status', 'tags',
                                        'part', 'modified', 'snippet', 'rightcol'])
lenta_ru_df = lenta_ru_df[~lenta_ru_df['text'].isin(['', None])]
lenta_ru_df.head()

,url,title,lastmodtime,type,bloc,image_url,pubdate,text,topic
0,https://lenta.ru/extlink/2023/01/13/soldar/,Россия взяла под контроль Соледар.,1673611969,15,1,https://icdn.lenta.ru/images/2023/01/13/15/202...,1673611969,NaN,Общество/Россия
1,https://lenta.ru/extlink/2023/01/11/novi/,Шойгу назначил нового командующего объединенно...,1673456711,15,1,https://icdn.lenta.ru/images/2023/01/11/19/202...,1673456711,NaN,Общество/Россия
2,https://lenta.ru/extlink/2023/01/16/dronisevas/,Севастополь атаковали беспилотники ВСУ.,1673869937,15,1,https://icdn.lenta.ru/images/2023/01/16/14/202...,1673865240,NaN,Общество/Россия
3,https://lenta.ru/extlink/2023/01/02/makeevka/,Украина в новогоднюю ночь обстреляла здание ПТ...,1672658892,15,1,https://icdn.lenta.ru/images/2023/01/02/14/202...,1672658892,NaN,Общество/Россия
4,https://lenta.ru/extlink/2023/01/12/vipuskayte/,В России впервые обнаружен новый вариант корон...,1673510781,15,1,https://icdn.lenta.ru/images/2023/01/12/10/202...,1673510220,NaN,Общество/Россия


In [ ]:
rbc_df1 = pd.read_excel('drive/MyDrive/rbc_arts.xlsx')
rbc_df1 = rbc_df1.drop(columns=['Unnamed: 0'])

rbc_df2 = pd.read_excel('drive/MyDrive/rbc_econ_and_russia_arts.xlsx')
rbc_df2 = rbc_df2.drop(columns=['Unnamed: 0'])

rbc_df = pd.concat([rbc_df1, rbc_df2])

In [ ]:
rbc_df.rename(columns={'topic_id': 'bloc',
                       'content': 'text'},
               inplace=True)

rbc_df.drop(columns=['datetime'], inplace=True)

In [ ]:
lenta_ru_df = lenta_ru_df[rbc_df.columns]

## 1. Предобработка данных

In [ ]:
rbc_df[rbc_df.duplicated()]

,topic,url,title,text,bloc
5938,Строительство,https://realty.rbc.ru/news/6874f0f39a7947c670d...,"«Дом.РФ» отметил, что самый существенный спад ...",За первое полугодие 2025 года банки выдали поч...,6
5939,Строительство,https://realty.rbc.ru/news/693a93eb9a7947fbfe9...,савсавсф,NaN,6
5940,Строительство,https://realty.rbc.ru/news/6929879d9a7947bdef6...,В Санкт-Петербурге на торги выставили историче...,«Почта России» выставила на торги здание Главп...,6
5941,Строительство,https://realty.rbc.ru/news/683038619a7947b0369...,Троицкий собор Троице-Сергиева монастыря,NaN,6
5942,Строительство,https://realty.rbc.ru/news/69295b7b9a794710dd5...,Собянин утвердил план благоустройства столицы ...,Мэр Москвы Сергей Собянин утвердил план благоу...,6
...,...,...,...,...,...
3302,Экономика,https://www.rbc.ru/economics/06/09/2024/66d95a...,В России предложили создать супермаркеты иннов...,Россия может взять на вооружение опыт Китая по...,4
3304,Экономика,https://www.rbc.ru/rbcfreenews/671c0bd79a7947d...,Путин подписал закон о регулировании майнинга,Президент России Владимир Путин подписал закон...,4
3311,Экономика,https://www.rbc.ru/economics/17/09/2024/66e965...,В ЦБ вместо «переохлаждения» экономики допусти...,Негативный сценарий переохлаждения экономики в...,4
3312,Экономика,https://www.rbc.ru/economics/16/09/2024/66e406...,Минэк спрогнозировал торможение роста в обраба...,В 2025 году темпы роста промышленного производ...,4


In [ ]:
def preprocessing(df: pd.DataFrame):
  df_copy = df.copy(deep=True)
  df_copy = df_copy.drop_duplicates() # удаляем дупликаты
  df_copy = df_copy.drop_duplicates(subset=['text'])  # пустые тексты
  df_copy = df_copy.dropna(subset=['text']) # повторы текстов
  df_copy = df_copy[~(df_copy['text'].str.split().str.len() < 50)] # тексты из менее, чем 50 слов
  df_copy = df_copy.reset_index(drop=True)
  return df_copy

In [ ]:
print(rbc_df.shape, preprocessing(rbc_df).shape)
rbc_df = preprocessing(rbc_df)

(13079, 5) (12197, 5)


In [ ]:
print(lenta_ru_df.shape, preprocessing(lenta_ru_df).shape)
lenta_ru_df = preprocessing(lenta_ru_df)

(28736, 5) (26854, 5)


In [ ]:
@np.vectorize
def remove_metadata(text: str) -> str:
  '''
    Deletes metadata such as "/ photo from Chunga changa /", and "Фото: {word} {word}".
  '''
  patterns = [
              re.compile(r'(?:\w+\s*){,2}\s*/\s*(?:\w+\s*)/?', flags=re.IGNORECASE),
              re.compile(r'Фото:(\s*(?:[\w.-]*\/?)){2}', flags=re.IGNORECASE),
              ]
  new_text = text
  for pattern in patterns:
    new_text = re.sub(pattern, '', new_text)
  return new_text.strip()


with Pool(5) as pool:
  lenta_ru_df['text'] = pool.map(remove_metadata, lenta_ru_df['text'])
lenta_ru_df['text'] = lenta_ru_df['text'].apply(str)

# уберем лишние приписки, которые не несут смысла, а являются лишь справочной информацией
# у рбк таких проблем нет

Process ForkPoolWorker-6:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.12/dist-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.12/dist-packages/multiprocess/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-package

KeyboardInterrupt: 

In [ ]:
arts_df = pd.concat([lenta_ru_df, rbc_df])
arts_df

,topic,url,title,text,bloc
7,Общество/Россия,https://lenta.ru/news/2023/01/24/leopard_peskov/,Кремль высказался о последствиях возможных пос...,Дмитров Возможные поставки Германией Украине т...,1
8,Общество/Россия,https://lenta.ru/news/2023/01/18/bespilot/,Российские военные уничтожили семь украинских ...,Родикова Российские военнослужащие уничтожили ...,1
9,Общество/Россия,https://lenta.ru/news/2023/01/20/tanki_minus/,Кремль предрек «последствия со знаком минус» и...,Если министры обороны стран Запада во время вс...,1
10,Общество/Россия,https://lenta.ru/news/2023/01/30/koronavirus/,Минобороны заявило о работе специалистов США н...,Родикова Специалисты американской компании Eco...,1
11,Общество/Россия,https://lenta.ru/news/2023/01/28/car/,На внутренней стороне МКАД загорелся автомобиль,«Дептранс. Оперативно» Марина Совина В Москве ...,1
...,...,...,...,...,...
9759,Туризм/Путешествия,https://rt.rbc.ru/tatarstan/freenews/67a066bc9...,"Казань в прошлом году посетили 4,5 млн туристов","В 2024 году Казань посетили 4,5 млн туристов, ...",48
9760,Туризм/Путешествия,https://karelia.rbc.ru/karelia/24/04/2025/6809...,Карелия вошла в топ-20 популярных маршрутов на...,Карелия попала в топ-20 популярных направлений...,48
9761,Туризм/Путешествия,https://rt.rbc.ru/tatarstan/05/11/2024/672622c...,Из Поднебесной в Татарстан: в республике расте...,Растущее число туристов из Китая Турпоток из К...,48
9762,Туризм/Путешествия,https://chr.rbc.ru/chr/freenews/686cd8049a7947...,Липецкий предприниматель оштрафован за использ...,Арбитражный суд Липецкой области рассмотрел ис...,48


In [ ]:
arts_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36230 entries, 7 to 9763
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   topic   36230 non-null  object
 1   url     36230 non-null  object
 2   title   36230 non-null  object
 3   text    36230 non-null  object
 4   bloc    36230 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


Найдем соответствие между кодом блока, его названием и кодом в соревновании:

* 1 - Россия - 0
* 37 - Силовые структуры - 2
* 3 - Бывший СССР - 3
* 4 - Экономика - 1
* 5 - Наука и техника - 8
* 8 - Спорт - 4
* 48 - Туризм - 7
* 87 - Здоровье - 5

In [ ]:
topic_name_to_lenta_id = {
    'Россия': 1,
    'Силовые структуры': 37,
    'Бывший СССР': 3,
    'Экономика': 4,
    'Наука и техника': 5,
    'Спорт': 8,
    'Туризм': 48,
    'Здоровье': 87,
    'Строительство': 6,
}

In [ ]:
arts_df_new = arts_df[arts_df.bloc.isin([1, 37, 3, 4, 5, 8, 48, 87, 6])]

TagsMap = {1 : 0, 3 : 3, 4 : 1, 5 : 8, 8 : 4, 37 : 2, 48 : 7, 87 : 5,
           6 : 6}


arts_df_new.loc[:, 'topic'] = arts_df_new['bloc'].map(TagsMap)

In [ ]:
arts_df_new.loc[:, 'topic'].unique()

array([0, 1, 2, 3, 4, 5, 7, 8, 6], dtype=object)

In [ ]:
pd.concat([arts_df_new.value_counts('topic'),
           arts_df_new.value_counts('topic', normalize=True)], axis=1)

,count,proportion
topic,,
1,5388,0.148717
7,5296,0.146177
0,5233,0.144438
4,4338,0.119735
3,3736,0.103119
8,3538,0.097654
2,3519,0.097129
5,3485,0.096191
6,1697,0.046840


## 2. Машинное обучение

Загружаем данные и обучаем модель на разбиении трейн-тест

In [ ]:
from string import punctuation
# удаление пунктуации из текста
def clean_from_punct(text: str) -> str:
    return re.sub(rf'[{punctuation}]', '', text)

In [ ]:
X = arts_df_new[['text']]
X.loc[:, 'text'] = X['text'].apply(clean_from_punct)
y = arts_df_new['topic'].astype(int)

X.shape, y.shape

((36230, 1), (36230,))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((27172, 1), (9058, 1))

### Baseline-решение

In [ ]:
baseline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('scaler', MaxAbsScaler()),
    ('model', LogisticRegression(max_iter=200, random_state=42, class_weight='balanced'))
])

baseline.fit(X_train['text'], y_train)
pred = baseline.predict(X_test['text'])

print(classification_report(y_test, pred))

Для векторизации текста в качестве бейзлайна случае я использую CountVectorizer, то есть метод Bag of words. Это значит, что находятся уникальные слова (токены), а потом для каждого наблюдения (текста) вычисляется частота этого токена. Поэтому кол-во столбцов = кол-ву уникальных токенов.

In [ ]:
pipe = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('scaler', MaxAbsScaler()),
    ('model', LogisticRegression(max_iter=200, random_state=42))
])


param_grid = [
    {
        'model': [LogisticRegression(max_iter=200,
                                     random_state=42,
                                     class_weight='balanced'),
                  KNeighborsClassifier(3)],

        'vectorizer': [CountVectorizer(),
                       TfidfVectorizer()],

        'vectorizer__ngram_range': [(1, 1)],
        # 'model__max_iter': [100, 200]
    }
]

clf = GridSearchCV(pipe, param_grid, cv=5, n_jobs=5)

Далее я добавляю метод TF-IDF (term frequency - inverse document frequency). Это отношение частоты вхождения токена в документ и логарифма обратной частоты документов, в которых встречается токен.

В качестве дополнительной модели классификации был применен метод k ближайших соседей.

In [ ]:
clf.fit(X_train['text'], y_train)

NameError: name 'clf' is not defined

In [ ]:
import pickle
with open('best_model.pkl', 'wb') as file:
    print(clf.best_estimator_, clf.best_params_)
    pickle.dump([clf.best_estimator_, clf.best_params_], file)

In [ ]:
pred = clf.predict(X_test['text']) # grid search

print(classification_report(y_test, pred))

Загружаем тестовые данные, обучаем итоговую модель и делаем прогноз.

In [ ]:
Test = pd.read_csv("drive/MyDrive/test_news.csv")
with Pool(5) as pool:
  Test['content'] = pool.map(remove_metadata, Test['content'])
Test['content'] = Test['content'].apply(str)

Test.head()

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [ ]:
pred = pipe.predict(Test['content'])
subm = pd.read_csv("base_submission_news.csv")
subm['topic'] = pred

subm.to_csv("bow_logreg.csv", index=False)

Лучший вариант - без удаления метаданных, пунктуации